In [58]:
from data_loader import Hotdog_NotHotdog
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# accumulate mean/std efficiently
def calc_mean_and_std(loader):
    mean = 0.
    std = 0.
    n_pix = 0
    n_images = 0
    for images, _ in tqdm(loader, desc="Calculating mean/std"):
        #images = images.to("mps")
        b, c, h, w = images.shape  # batch, channels, height, width
        n_pixels = b * h * w

        # sum over batch, keep channel dimension
        mean += images.sum(dim=[0, 2, 3])
        std  += (images ** 2).sum(dim=[0, 2, 3])
        n_pix += n_pixels
        n_images += b
    # print("mean: {}, n_pix: {}, n_images: {}".format(mean, n_pix, n_images))
    mean /= n_pix
    std = (std / n_pix - mean ** 2).sqrt()
    return mean, std



In [54]:
# Step 1: basic transform, only ToTensor() (NO normalization, NO augmentation!)
img_size = 224
stat_tfms = transforms.Compose([
    transforms.Resize(256, antialias=True),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),                 # values in [0,1]
])
# Step 2: load dataset (train split only)
batch_size = 64
data_path = "../data/hotdog_nothotdog"

In [56]:
trainset = Hotdog_NotHotdog(train=True, transform=stat_tfms, data_path=data_path)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=False, num_workers=4)

train_mean, train_std = calc_mean_and_std(train_loader)
# print("dataset length: {}".format(len(trainset)))

print("Training Dataset mean: {}, std: {}".format(train_mean.tolist(), train_std.tolist()))

Calculating mean/std:  16%|█▌        | 5/32 [00:02<00:08,  3.04it/s]

b,c,h,w: 64, 3, 224, 224


Calculating mean/std: 100%|██████████| 32/32 [00:23<00:00,  1.39it/s]

mean: tensor([55714384., 45562164., 35374720.]), n_pix: 102710272, n_images: 2047
Training Dataset mean: [0.5424422025680542, 0.4435988962650299, 0.3444126844406128], std: [0.26526492834091187, 0.2597216069698334, 0.2690683901309967]


In [59]:
testset = Hotdog_NotHotdog(train=False, transform=stat_tfms, data_path=data_path)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

test_mean, test_std = calc_mean_and_std(test_loader)

print("Testing Dataset mean: {}, std: {}".format(test_mean.tolist(), test_std.tolist()))

Calculating mean/std: 100%|██████████| 30/30 [00:22<00:00,  1.32it/s]

Testing Dataset mean: [0.543861448764801, 0.4478202760219574, 0.3497999906539917], std: [0.263683021068573, 0.25815925002098083, 0.26918601989746094]
